# 🛑 Gestion des Stopwords Français

**Module 2 - Preprocessing et Tokenisation**

Dans ce notebook, nous allons explorer en détail la gestion des stopwords (mots vides) en français, comparer les approches NLTK et spaCy, et créer nos propres listes personnalisées.

## 📋 Plan du Notebook

1. **Introduction aux Stopwords**
2. **Stopwords avec NLTK**
3. **Stopwords avec spaCy**
4. **Comparaison NLTK vs spaCy**
5. **Stopwords Personnalisés**
6. **Cas Pratiques**
7. **Visualisations et Analyses**
8. **Bonnes Pratiques**

## 📦 Installation et Imports

In [ ]:
# Installation des packages nécessaires
# !pip install nltk spacy matplotlib seaborn wordcloud
# !python -m spacy download fr_core_news_sm

In [ ]:
import nltk
import spacy
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from wordcloud import WordCloud
import re
from typing import List, Set, Dict

# Configuration pour les graphiques
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✅ Imports réalisés avec succès !")

## 1. 🎯 Introduction aux Stopwords

### Qu'est-ce qu'un stopword ?

Un **stopword** (mot vide) est un mot très fréquent dans une langue mais qui apporte peu d'information sémantique pour l'analyse de texte.

### Exemples en français :
- **Articles** : le, la, les, un, une, des
- **Prépositions** : de, à, dans, pour, avec, sur
- **Pronoms** : je, tu, il, elle, nous, vous, ils, elles
- **Conjonctions** : et, ou, mais, car, donc
- **Auxiliaires** : être, avoir

In [ ]:
# Exemple d'impact des stopwords
texte_exemple = """
Le chat noir mange des croquettes avec grand appétit dans le jardin ensoleillé. 
Il est très content et ronronne de bonheur. Les oiseaux chantent dans les arbres 
pendant que le soleil brille de mille feux.
"""

# Tokenisation simple
mots = re.findall(r'\w+', texte_exemple.lower())
print(f"📝 Texte original : {len(mots)} mots")
print(f"Mots : {mots[:15]}...\n")

# Comptage des fréquences
frequences = Counter(mots)
print("🔝 Top 10 des mots les plus fréquents :")
for mot, freq in frequences.most_common(10):
    print(f"  '{mot}' : {freq} fois")

### 💡 Observation

Comme vous pouvez le voir, les mots les plus fréquents sont souvent des mots grammaticaux peu informatifs. C'est exactement le problème que les stopwords permettent de résoudre !

## 2. 🐍 Stopwords avec NLTK

In [ ]:
# Téléchargement des ressources NLTK
nltk.download('stopwords', quiet=True)
from nltk.corpus import stopwords

# Obtenir les stopwords français
stopwords_nltk = set(stopwords.words('french'))

print(f"📊 NLTK fournit {len(stopwords_nltk)} stopwords français")
print(f"\n🔤 Échantillon de stopwords NLTK :")
print(sorted(list(stopwords_nltk))[:20])

In [ ]:
# Fonction pour supprimer les stopwords avec NLTK
def supprimer_stopwords_nltk(texte: str) -> List[str]:
    """
    Supprime les stopwords d'un texte en utilisant NLTK.
    
    Args:
        texte: Texte à traiter
    
    Returns:
        Liste des mots sans stopwords
    """
    mots = re.findall(r'\w+', texte.lower())
    mots_filtres = [mot for mot in mots if mot not in stopwords_nltk]
    return mots_filtres

# Test sur notre exemple
mots_sans_stopwords_nltk = supprimer_stopwords_nltk(texte_exemple)

print(f"📊 Statistiques avec NLTK :")
print(f"  Mots originaux : {len(mots)}")
print(f"  Mots après filtrage : {len(mots_sans_stopwords_nltk)}")
print(f"  Réduction : {(1 - len(mots_sans_stopwords_nltk)/len(mots))*100:.1f}%")
print(f"\n🎯 Mots conservés : {mots_sans_stopwords_nltk[:15]}")

## 3. ⚡ Stopwords avec spaCy

In [ ]:
# Chargement du modèle français de spaCy
try:
    nlp = spacy.load("fr_core_news_sm")
    print("✅ Modèle spaCy français chargé avec succès")
except OSError:
    print("❌ Modèle spaCy français non trouvé")
    print("   Exécutez : python -m spacy download fr_core_news_sm")
    # Alternative : utiliser le modèle de base
    nlp = spacy.load("fr_core_news_sm")

# Obtenir les stopwords de spaCy
stopwords_spacy = nlp.Defaults.stop_words

print(f"\n📊 spaCy fournit {len(stopwords_spacy)} stopwords français")
print(f"\n🔤 Échantillon de stopwords spaCy :")
print(sorted(list(stopwords_spacy))[:20])

In [ ]:
# Fonction pour supprimer les stopwords avec spaCy
def supprimer_stopwords_spacy(texte: str) -> List[str]:
    """
    Supprime les stopwords d'un texte en utilisant spaCy.
    
    Args:
        texte: Texte à traiter
    
    Returns:
        Liste des mots sans stopwords
    """
    doc = nlp(texte.lower())
    mots_filtres = [token.text for token in doc 
                   if not token.is_stop and not token.is_punct and not token.is_space]
    return mots_filtres

# Test sur notre exemple
mots_sans_stopwords_spacy = supprimer_stopwords_spacy(texte_exemple)

print(f"📊 Statistiques avec spaCy :")
print(f"  Mots originaux : {len(mots)}")
print(f"  Mots après filtrage : {len(mots_sans_stopwords_spacy)}")
print(f"  Réduction : {(1 - len(mots_sans_stopwords_spacy)/len(mots))*100:.1f}%")
print(f"\n🎯 Mots conservés : {mots_sans_stopwords_spacy[:15]}")

## 4. ⚔️ Comparaison NLTK vs spaCy

In [ ]:
# Analyse comparative des listes de stopwords
print("🔍 ANALYSE COMPARATIVE DES STOPWORDS\n")

# Statistiques de base
print(f"📊 Tailles des listes :")
print(f"  NLTK : {len(stopwords_nltk)} mots")
print(f"  spaCy : {len(stopwords_spacy)} mots")

# Intersections et différences
communs = stopwords_nltk.intersection(stopwords_spacy)
uniquement_nltk = stopwords_nltk - stopwords_spacy
uniquement_spacy = stopwords_spacy - stopwords_nltk

print(f"\n🤝 Mots communs : {len(communs)}")
print(f"🐍 Uniquement NLTK : {len(uniquement_nltk)}")
print(f"⚡ Uniquement spaCy : {len(uniquement_spacy)}")

print(f"\n📝 Exemples de différences :")
print(f"  Uniquement NLTK : {sorted(list(uniquement_nltk))[:10]}")
print(f"  Uniquement spaCy : {sorted(list(uniquement_spacy))[:10]}")

In [ ]:
# Visualisation comparative
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Graphique en secteurs des intersections
labels = ['Communs', 'NLTK seul', 'spaCy seul']
sizes = [len(communs), len(uniquement_nltk), len(uniquement_spacy)]
colors = ['#ff9999', '#66b3ff', '#99ff99']

axes[0].pie(sizes, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
axes[0].set_title('🥧 Répartition des Stopwords', fontsize=14, fontweight='bold')

# Comparaison des longueurs de listes
libraries = ['NLTK', 'spaCy', 'Communs']
counts = [len(stopwords_nltk), len(stopwords_spacy), len(communs)]
colors_bar = ['#ff7f0e', '#2ca02c', '#d62728']

bars = axes[1].bar(libraries, counts, color=colors_bar, alpha=0.7)
axes[1].set_title('📊 Nombre de Stopwords par Bibliothèque', fontsize=14, fontweight='bold')
axes[1].set_ylabel('Nombre de stopwords')

# Ajout des valeurs sur les barres
for bar, count in zip(bars, counts):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1, 
                str(count), ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Comparaison sur plusieurs textes types
textes_test = {
    "Article de presse": """
    Le gouvernement français a annoncé hier de nouvelles mesures économiques. 
    Ces décisions, qui entreront en vigueur dès le mois prochain, visent à 
    stimuler la croissance et à réduire le chômage dans les régions les plus touchées.
    """,
    
    "Avis client": """
    Ce produit est vraiment génial ! Je le recommande vivement à tous ceux qui 
    cherchent quelque chose de fiable et d'efficace. Le service client est également 
    très réactif et professionnel. Un excellent achat !
    """,
    
    "Tweet informel": """
    Salut tout le monde ! Alors, qu'est-ce que vous pensez du nouveau film ? 
    Moi je l'ai trouvé plutôt pas mal, mais bon, c'est pas non plus extraordinaire...
    """
}

resultats_comparaison = []

for titre, texte in textes_test.items():
    mots_originaux = re.findall(r'\w+', texte.lower())
    mots_nltk = supprimer_stopwords_nltk(texte)
    mots_spacy = supprimer_stopwords_spacy(texte)
    
    resultats_comparaison.append({
        'Type': titre,
        'Mots originaux': len(mots_originaux),
        'Après NLTK': len(mots_nltk),
        'Après spaCy': len(mots_spacy),
        'Réduction NLTK (%)': round((1 - len(mots_nltk)/len(mots_originaux))*100, 1),
        'Réduction spaCy (%)': round((1 - len(mots_spacy)/len(mots_originaux))*100, 1)
    })

df_comparaison = pd.DataFrame(resultats_comparaison)
print("📋 COMPARAISON SUR DIFFÉRENTS TYPES DE TEXTES\n")
print(df_comparaison.to_string(index=False))

## 5. 🎯 Stopwords Personnalisés

Parfois, les listes prédéfinies ne suffisent pas. Vous devez créer vos propres listes selon :
- **Le domaine** : médical, juridique, technique...
- **Le type de texte** : tweets, articles, emails...
- **L'objectif** : sentiment, classification, extraction d'entités...

In [ ]:
# Création de listes de stopwords personnalisées par domaine

stopwords_domaines = {
    'e-commerce': {
        'produit', 'article', 'commande', 'livraison', 'client', 'service', 
        'achat', 'vente', 'prix', 'euros', 'boutique', 'magasin', 'site',
        'avis', 'commentaire', 'note', 'étoiles'
    },
    
    'médical': {
        'patient', 'médecin', 'docteur', 'traitement', 'maladie', 'symptôme',
        'diagnostic', 'hôpital', 'clinique', 'ordonnance', 'médicament',
        'consultation', 'examen', 'analyse'
    },
    
    'réseaux_sociaux': {
        'mdr', 'lol', 'ptdr', 'jsp', 'slt', 'bjr', 'bsr', 'cv', 'oui', 'non',
        'ouais', 'genre', 'style', 'truc', 'machin', 'chose', 'ça', 'ca'
    },
    
    'actualités': {
        'selon', 'également', 'notamment', 'toutefois', 'cependant', 'néanmoins',
        'par ailleurs', 'en effet', 'ainsi', 'donc', 'enfin', 'finalement'
    }
}

print("🎯 STOPWORDS PERSONNALISÉS PAR DOMAINE\n")
for domaine, mots in stopwords_domaines.items():
    print(f"📂 {domaine.upper()} ({len(mots)} mots) :")
    print(f"   {', '.join(sorted(list(mots))[:8])}...\n")

In [ ]:
# Classe pour gérer les stopwords personnalisés
class StopwordsPersonnalises:
    def __init__(self, base='nltk'):
        """
        Initialise avec une base de stopwords (nltk ou spacy).
        """
        if base == 'nltk':
            self.stopwords = set(stopwords.words('french'))
        elif base == 'spacy':
            self.stopwords = nlp.Defaults.stop_words.copy()
        else:
            self.stopwords = set()
    
    def ajouter_domaine(self, domaine: str) -> None:
        """
        Ajoute les stopwords d'un domaine spécifique.
        """
        if domaine in stopwords_domaines:
            self.stopwords.update(stopwords_domaines[domaine])
            print(f"✅ Stopwords du domaine '{domaine}' ajoutés")
        else:
            print(f"❌ Domaine '{domaine}' non reconnu")
    
    def ajouter_mots(self, mots: List[str]) -> None:
        """
        Ajoute une liste de mots personnalisés.
        """
        self.stopwords.update(mots)
        print(f"✅ {len(mots)} mots ajoutés aux stopwords")
    
    def supprimer_mots(self, mots: List[str]) -> None:
        """
        Supprime des mots de la liste des stopwords.
        """
        self.stopwords -= set(mots)
        print(f"✅ {len(mots)} mots supprimés des stopwords")
    
    def filtrer_texte(self, texte: str) -> List[str]:
        """
        Filtre un texte en supprimant les stopwords.
        """
        mots = re.findall(r'\w+', texte.lower())
        return [mot for mot in mots if mot not in self.stopwords]
    
    def statistiques(self) -> Dict:
        """
        Retourne des statistiques sur les stopwords.
        """
        return {
            'total': len(self.stopwords),
            'longueur_moyenne': sum(len(mot) for mot in self.stopwords) / len(self.stopwords),
            'mot_plus_long': max(self.stopwords, key=len),
            'mot_plus_court': min(self.stopwords, key=len)
        }

# Exemple d'utilisation
print("🛠️ CRÉATION D'UN GESTIONNAIRE PERSONNALISÉ\n")

# Gestionnaire pour e-commerce
stopwords_ecommerce = StopwordsPersonnalises(base='nltk')
stopwords_ecommerce.ajouter_domaine('e-commerce')
stopwords_ecommerce.ajouter_mots(['vraiment', 'super', 'très', 'assez', 'plutôt'])

print(f"\n📊 Statistiques : {stopwords_ecommerce.statistiques()}")

## 6. 🧪 Cas Pratiques

Testons nos différentes approches sur des textes réels de différents domaines.

In [ ]:
# Textes d'exemple pour les cas pratiques
cas_pratiques = {
    "Avis e-commerce": """
    Ce produit est vraiment génial ! La livraison a été super rapide et le service client 
    très réactif. Je recommande vivement cet article à tous ceux qui cherchent de la qualité. 
    Excellent rapport qualité prix ! 5 étoiles sans hésiter.
    """,
    
    "Tweet informel": """
    Salut les amis ! Alors, qu'est-ce que vous pensez du dernier épisode ? 
    Moi jsp, j'ai trouvé ça plutôt bizarre... Genre, c'était pas mal mais bon, 
    ça manquait un peu de rythme non ? Bref, dites-moi vos avis !
    """,
    
    "Article médical": """
    Le patient présente des symptômes caractéristiques de cette pathologie. 
    Le médecin recommande un traitement adapté avec surveillance régulière. 
    L'examen clinique révèle des signes compatibles avec le diagnostic initial.
    """
}

# Test avec différentes configurations
configurations = {
    'Standard NLTK': StopwordsPersonnalises(base='nltk'),
    'Standard spaCy': StopwordsPersonnalises(base='spacy'),
    'E-commerce': StopwordsPersonnalises(base='nltk'),
    'Réseaux sociaux': StopwordsPersonnalises(base='nltk'),
    'Médical': StopwordsPersonnalises(base='nltk')
}

# Configuration des gestionnaires spécialisés
configurations['E-commerce'].ajouter_domaine('e-commerce')
configurations['Réseaux sociaux'].ajouter_domaine('réseaux_sociaux')
configurations['Médical'].ajouter_domaine('médical')

print("🧪 CAS PRATIQUES - COMPARAISON DES APPROCHES\n")

for titre_texte, texte in cas_pratiques.items():
    print(f"📄 {titre_texte.upper()}")
    print(f"Texte : {texte.strip()[:100]}...\n")
    
    mots_originaux = re.findall(r'\w+', texte.lower())
    
    for nom_config, gestionnaire in configurations.items():
        mots_filtres = gestionnaire.filtrer_texte(texte)
        reduction = (1 - len(mots_filtres)/len(mots_originaux)) * 100
        
        print(f"  {nom_config:20s} : {len(mots_filtres):2d} mots ({reduction:5.1f}% réduction)")
        print(f"  {'':22s} → {mots_filtres[:8]}")
    
    print("\n" + "="*60 + "\n")

## 7. 📊 Visualisations et Analyses

In [ ]:
# Analyse de l'impact des stopwords sur un corpus plus large
corpus_exemple = [
    "Ce produit est vraiment excellent, je le recommande vivement à tous mes amis.",
    "Service client décevant, j'ai attendu plus d'une heure au téléphone sans réponse.",
    "La livraison était rapide mais l'emballage était abîmé à l'arrivée.",
    "Très satisfait de mon achat, la qualité est au rendez-vous comme toujours.",
    "Prix un peu élevé mais la qualité justifie largement cet investissement.",
    "Interface utilisateur intuitive et design moderne, parfait pour mon usage quotidien.",
    "Problème technique récurrent, le support n'arrive pas à résoudre le dysfonctionnement.",
    "Excellente alternative aux produits concurrents, rapport qualité-prix imbattable."
]

# Analyse avant/après pour chaque phrase
resultats_analyse = []

for i, phrase in enumerate(corpus_exemple):
    mots_originaux = re.findall(r'\w+', phrase.lower())
    mots_sans_stopwords = supprimer_stopwords_nltk(phrase)
    
    resultats_analyse.append({
        'Phrase': i + 1,
        'Mots_originaux': len(mots_originaux),
        'Mots_filtres': len(mots_sans_stopwords),
        'Reduction_pct': (1 - len(mots_sans_stopwords)/len(mots_originaux)) * 100
    })

df_analyse = pd.DataFrame(resultats_analyse)

# Visualisation
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Graphique 1: Réduction par phrase
axes[0,0].bar(df_analyse['Phrase'], df_analyse['Reduction_pct'], 
              color='skyblue', alpha=0.7, edgecolor='darkblue')
axes[0,0].set_title('📉 Réduction du vocabulaire par phrase', fontweight='bold')
axes[0,0].set_xlabel('Numéro de phrase')
axes[0,0].set_ylabel('Réduction (%)')
axes[0,0].grid(True, alpha=0.3)

# Graphique 2: Comparaison avant/après
x = df_analyse['Phrase']
width = 0.35
axes[0,1].bar(x - width/2, df_analyse['Mots_originaux'], width, 
              label='Avant', color='lightcoral', alpha=0.7)
axes[0,1].bar(x + width/2, df_analyse['Mots_filtres'], width, 
              label='Après', color='lightgreen', alpha=0.7)
axes[0,1].set_title('📊 Nombre de mots avant/après filtrage', fontweight='bold')
axes[0,1].set_xlabel('Numéro de phrase')
axes[0,1].set_ylabel('Nombre de mots')
axes[0,1].legend()
axes[0,1].grid(True, alpha=0.3)

# Graphique 3: Distribution des réductions
axes[1,0].hist(df_analyse['Reduction_pct'], bins=6, color='gold', alpha=0.7, 
               edgecolor='darkorange', rwidth=0.8)
axes[1,0].set_title('📈 Distribution des taux de réduction', fontweight='bold')
axes[1,0].set_xlabel('Réduction (%)')
axes[1,0].set_ylabel('Nombre de phrases')
axes[1,0].grid(True, alpha=0.3)

# Graphique 4: Statistiques globales
stats = ['Mots totaux\navant', 'Mots totaux\naprès', 'Réduction\nmoyenne (%)']
values = [df_analyse['Mots_originaux'].sum(), 
          df_analyse['Mots_filtres'].sum(),
          df_analyse['Reduction_pct'].mean()]
colors = ['lightblue', 'lightgreen', 'lightyellow']

bars = axes[1,1].bar(stats, values, color=colors, alpha=0.7, 
                     edgecolor=['darkblue', 'darkgreen', 'orange'])
axes[1,1].set_title('📋 Statistiques globales', fontweight='bold')

# Ajout des valeurs sur les barres
for bar, value in zip(bars, values):
    height = bar.get_height()
    axes[1,1].text(bar.get_x() + bar.get_width()/2., height + max(values)*0.01,
                   f'{value:.1f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

print("📊 RÉSUMÉ DE L'ANALYSE :")
print(f"  💡 Réduction moyenne : {df_analyse['Reduction_pct'].mean():.1f}%")
print(f"  📉 Réduction minimale : {df_analyse['Reduction_pct'].min():.1f}%")
print(f"  📈 Réduction maximale : {df_analyse['Reduction_pct'].max():.1f}%")
print(f"  🎯 Mots économisés : {df_analyse['Mots_originaux'].sum() - df_analyse['Mots_filtres'].sum()}")

In [ ]:
# Nuage de mots des stopwords les plus courants
try:
    # Combinaison des stopwords NLTK et spaCy
    tous_stopwords = stopwords_nltk.union(stopwords_spacy)
    
    # Création du nuage de mots
    texte_stopwords = ' '.join(tous_stopwords)
    
    wordcloud = WordCloud(
        width=800, height=400, 
        background_color='white',
        colormap='viridis',
        max_words=100,
        relative_scaling=0.5,
        random_state=42
    ).generate(texte_stopwords)
    
    plt.figure(figsize=(12, 6))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis('off')
    plt.title('☁️ Nuage de Mots des Stopwords Français', 
              fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.show()
    
except ImportError:
    print("❌ WordCloud non disponible. Installez avec : pip install wordcloud")

## 8. 💡 Bonnes Pratiques

### ✅ À FAIRE

1. **Adapter au contexte** : Utiliser des stopwords spécifiques au domaine
2. **Tester l'impact** : Mesurer l'effet sur les performances finales
3. **Documenter les choix** : Justifier pourquoi tel mot est considéré comme stopword
4. **Être itératif** : Ajuster la liste selon les résultats obtenus
5. **Conserver l'original** : Toujours garder une copie du texte non filtré

### ❌ À ÉVITER

1. **Sur-filtrage** : Supprimer trop de mots importants
2. **One-size-fits-all** : Utiliser la même liste pour tous les cas
3. **Ignorer le contexte** : Un mot peut être stopword dans un cas mais pas dans un autre
4. **Oublier la casse** : Gérer les majuscules/minuscules
5. **Ne pas valider** : Vérifier manuellement les résultats sur un échantillon

In [ ]:
# Fonction utilitaire finale pour l'analyse de stopwords
def analyser_impact_stopwords(texte: str, stopwords_list: Set[str] = None) -> Dict:
    """
    Analyse l'impact de la suppression des stopwords sur un texte.
    
    Args:
        texte: Texte à analyser
        stopwords_list: Liste des stopwords (défaut: NLTK français)
    
    Returns:
        Dictionnaire avec les statistiques d'impact
    """
    if stopwords_list is None:
        stopwords_list = set(stopwords.words('french'))
    
    # Tokenisation
    mots_originaux = re.findall(r'\w+', texte.lower())
    mots_filtres = [mot for mot in mots_originaux if mot not in stopwords_list]
    
    # Identification des stopwords supprimés
    stopwords_trouves = [mot for mot in mots_originaux if mot in stopwords_list]
    
    # Calcul des statistiques
    reduction_pct = (1 - len(mots_filtres)/len(mots_originaux)) * 100 if mots_originaux else 0
    
    return {
        'mots_originaux': len(mots_originaux),
        'mots_apres_filtrage': len(mots_filtres),
        'stopwords_supprimes': len(stopwords_trouves),
        'reduction_pourcentage': round(reduction_pct, 1),
        'stopwords_uniques': len(set(stopwords_trouves)),
        'mots_conserves': mots_filtres[:10],  # Échantillon
        'stopwords_liste': list(set(stopwords_trouves))[:10]  # Échantillon
    }

# Test de la fonction
texte_test = """
Le service client de cette entreprise est vraiment excellent. J'ai été très 
satisfait de ma commande et je recommande vivement ce produit à tous ceux qui 
cherchent de la qualité. Le rapport qualité-prix est imbattable !
"""

analyse = analyser_impact_stopwords(texte_test)

print("🔍 ANALYSE DÉTAILLÉE D'IMPACT\n")
print(f"📊 Statistiques :")
print(f"  • Mots originaux : {analyse['mots_originaux']}")
print(f"  • Mots après filtrage : {analyse['mots_apres_filtrage']}")
print(f"  • Stopwords supprimés : {analyse['stopwords_supprimes']}")
print(f"  • Réduction : {analyse['reduction_pourcentage']}%")
print(f"  • Stopwords uniques trouvés : {analyse['stopwords_uniques']}")

print(f"\n🎯 Échantillon de mots conservés :")
print(f"  {', '.join(analyse['mots_conserves'])}")

print(f"\n🛑 Stopwords trouvés dans le texte :")
print(f"  {', '.join(analyse['stopwords_liste'])}")

## 🎉 Conclusion

### Ce que vous avez appris :

✅ **Comprendre** le rôle des stopwords dans le preprocessing  
✅ **Utiliser** NLTK et spaCy pour la gestion des stopwords français  
✅ **Comparer** les différentes approches et leurs avantages  
✅ **Créer** des listes personnalisées selon le domaine  
✅ **Analyser** l'impact des stopwords sur vos données  
✅ **Visualiser** les résultats et optimiser vos choix  

### Prochaines étapes :

🎯 **Lemmatisation et Stemming** : Réduction des mots à leur forme canonique  
🎯 **Pipeline complet** : Intégration de toutes les techniques de preprocessing  
🎯 **Évaluation** : Mesurer l'impact sur les performances des modèles  

---

**💡 Conseil final :** Les stopwords ne sont pas universels ! Adaptez toujours votre approche selon votre contexte, votre domaine et vos objectifs.